In [2]:
import pandas as pd
import duckdb
import os
import re

In [6]:
def all_pids():
    """
    Find list of all pids currently available.
    """
    pattern_current_DIR = "../cta-stop-etl/out/patterns_current"
    pattern_historic_DIR = "../cta-stop-etl/out/patterns_historic"

    pids_c = []
    for pid_file in os.listdir(pattern_current_DIR):

        if not pid_file.endswith(".parquet"):
            continue

        numbers = re.findall(r"\d+", pid_file)
        pid = numbers[0]
        pids_c.append(pid)

    pids_h = []
    for pid_file in os.listdir(pattern_historic_DIR):
        if not pid_file.endswith(".parquet"):
            continue

        numbers = re.findall(r"\d+", pid_file)
        pid = numbers[0]
        pids_h.append(pid)

    pids = set(pids_c + pids_h)

    return pids

In [16]:


pattern_pids = all_pids("../cta-stop-etl/out") 
pattern_pids_df  = pd.DataFrame(list(pattern_pids), columns=["pid"])
print(len(pattern_pids_df.index))

859


In [15]:

trips_DIR = "../cta-stop-etl/out/cta_bus_full_day_data_v2.parquet"
# combine with duckdb
cmd_number = f"""
SELECT cast(pid as int) as pid
FROM read_parquet('{trips_DIR}')
group by pid
"""

trip_pids = duckdb.execute(cmd_number).df()
print(len(trip_pids.index))

948


In [11]:
pattern_pids_df['pid_pattern'] = pattern_pids_df['pid'].astype(str)
trip_pids['pid'] = trip_pids['pid'].astype(str)

join_df = trip_pids.merge(pattern_pids_df, on='pid', how='left')

In [13]:
print(join_df.head())

join_df[join_df['pid_pattern'].isnull()]['pid']

     pid pid_pattern
0   1682        1682
1   7570        7570
2  10905       10905
3   8147        8147
4  10956       10956


59      5568
64      5431
65      8133
67     10908
68      6672
       ...  
892    10942
934     7386
937     2271
939     1043
943    19399
Name: pid, Length: 103, dtype: object